# Smart NIC with DPDK and Packet gen

DPDK is the Data Plane Development Kit that consists of libraries to accelerate packet processing workloads running on a wide variety of CPU architectures. It is designed to run on x86, POWER, and ARM processors and licensed under the Open-Source BSD License.

Pktgen-DPDK is a traffic generator powered by DPDK.

This notebook depicts how to attach Smart NICs to DPDK and send traffic to the DPDK connected NICs via Pktgen-DPDK.

## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

## Re-create a VM attached to dpdk-mellanox-tools volume on EDC

In order to have access to necessary tools execute the notebook to [re-create a Storage VM attached](./dpdk_tools_storage.ipynb) to the `dpdk-mellanox-tools` persistent storage. You must execute it as a member of project which owns the volume.

This step is optional. You can skip this step but would be then required to download Mellanox drivers from NVIDIA's [website](https://network.nvidia.com/products/infiniband-drivers/linux/mlnx_ofed/) and upload to the VMs. 

## Create the Experiment Slice

The following creates two nodes with NIC_ConnectX_6/NIC_ConnectX_5 NICs connected to an isolated local Ethernet. 
- `l3fwd-node`: with one port connected to local L2 network. We will run DPDK L3FWD application here
- `pktgen-node`: with one port connected to local L2 network. We wull run DPDK Packet Generator her 

In addition to local L2 network, each node is connected to FABNETv4 to be able to download the drivers from the Storage Node.

<img src="./images/dpdk-pktgen-smart-nic.png"><br>

In [ ]:
slice_name = 'MySlice-dpdk-pktgen'
#site = fablib.get_random_site()
site = "LBNL"
print(f"Site: {site}")

l3fwd_node_name = 'l3fwd-node'
pktgen_node_name = 'pktgen-node'

network_name='net1'

# username and password used in storage VM
nginx_user = "dpdk_mellanox_tools"
nginx_password = "secret-password"
storage_vm_ip = "10.129.1.2"

rocky_mellanox_iso_name = "MLNX_OFED_LINUX-23.07-0.5.0.0-rhel9.2-x86_64.iso"
ubuntu_mellanox_iso_name = "MLNX_OFED_LINUX-23.07-0.5.1.2-ubuntu20.04-x86_64.iso"


image_name = "default_rocky_9"
#image_name = "default_ubuntu_20"

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Network
net1 = slice.add_l2network(name=network_name, subnet=IPv4Network("192.168.1.0/24"))

# l3fwd_node
l3fwd_node = slice.add_node(name=l3fwd_node_name, site=site, cores=8, ram=32, disk=10, image=image_name)
l3fwd_node.add_fabnet()
iface1 = l3fwd_node.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]

# pktgen_node
pktgen_node = slice.add_node(name=pktgen_node_name, site=site, cores=8, ram=32, disk=10, image=image_name)
pktgen_node.add_fabnet()
iface2 = pktgen_node.add_component(model='NIC_ConnectX_6', name='nic1').get_interfaces()[0]

iface1.set_mode('auto')
iface2.set_mode('auto')

net1.add_interface(iface1)
net1.add_interface(iface2)

for n in slice.get_nodes():
    n.add_post_boot_upload_directory('node_tools','.')
    n.add_post_boot_execute('chmod +x node_tools/*')
    n.add_post_boot_execute('sudo node_tools/install.sh')

#Submit Slice Request
slice.submit();

## Verify connectivity between nodes on the local ethernet

In [ ]:
l3fwd_node = slice.get_node(name=l3fwd_node_name)        
pktgen_node = slice.get_node(name=pktgen_node_name)           

pktgen_node_addr = pktgen_node.get_interface(network_name=network_name).get_ip_addr()

stdout, stderr = l3fwd_node.execute(f'ping -c 5 {pktgen_node_addr}')

## Setup IOMMU and Hugepages 

For DPDK to function properly we need to setup IOMMU and hugepages on the VMs

In [ ]:
for n in slice.get_nodes():
    stdout, stderr = n.execute(f"sudo node_tools/grub.sh {n.get_ram()}", quiet=True, output_file=f"logs/{n.get_name()}-grub.log")
    stdout, stderr = n.execute("sudo node_tools/apply_vfio_settings.sh", quiet=True, output_file=f"logs/{n.get_name()}-vfio.log")

## Install Mellanox Drivers

Download the drivers from storage node and install. This step should be executed only if the Storage Node is available from where the drivers can be downloaded.
If the storage node is not available, the user is expected to download the Mellanox drivers from from NVIDIA's [website](https://network.nvidia.com/products/infiniband-drivers/linux/mlnx_ofed/) and upload to the VMs. User should then install the drivers using the steps:
```
sudo mount -o ro,loop <Driver ISO location> /mnt/
sudo /mnt/mlnxofedinstall --force
sudo /etc/init.d/openibd restart
```

In [ ]:
for n in slice.get_nodes():
    if "rocky" in n.get_image():
        mellanox_iso_name = rocky_mellanox_iso_name
    else:
        mellanox_iso_name = ubuntu_mellanox_iso_name
    stdout, stderr = n.execute(f"wget --no-check-certificate --user=dpdk_mellanox_tools --password=secret-password https://{storage_vm_ip}/dpdk-mellanox-tools/{mellanox_iso_name}", quiet=True)
    execute_threads[n] = n.execute_thread(f"sudo node_tools/fw.sh", output_file=f"logs/{n.get_name()}-drivers.log")
    
    
#Wait for results from threads
for n,thread in execute_threads.items():
    print(f"Waiting for result from node {n.get_name()}")
    stdout,stderr = thread.result()   
    
print("Driver install completed!")

## Reboot the VMs

In [ ]:
for n in slice.get_nodes():
    stdout, stderr = n.execute("sudo reboot", quiet=True)

## Wait for the VMs to be back up

In [ ]:
slice = fablib.get_slice(slice_name)
slice.wait_ssh()

In [ ]:
for n in slice.get_nodes():
    n.config()

## Download and build DPDK
Download the DPDK source code from DPDK website: https://fast.dpdk.org/rel/ and then build using the following commands.

The DPDK libraries and binaries will be installed at `/usr/local/lib64` and `/usr/local/bin/`.

In [ ]:
execute_threads = {}
for n in slice.get_nodes():
   execute_threads[n] = n.execute_thread("node_tools/dpdk.sh", output_file=f"logs/{n.get_name()}-dpdk.log")

#Wait for results from threads
for n,thread in execute_threads.items():
    print(f"Waiting for result from node {n.get_name()}")
    stdout,stderr = thread.result()  
    
print("DPDK install completed!")

## Download and build PKTGEN-DPDK

This section is only applied on the `pktgen-node`. Download the Pktgen-DPDK source code from github. As Pktgen-DPDK uses the DPDK libraries installed in prior section, please select a version which is close to the DPDK version. Then build using the following commands:

In [ ]:
pktgen_node = slice.get_node(name=pktgen_node_name)
stdout, stderr = pktgen_node.execute("node_tools/pktgen.sh", quiet=True, output_file=f"logs/{pktgen_node.get_name()}-dpdk-pktgen.log")

## Get the Nodes

In [ ]:
l3fwd_node = slice.get_node(name=l3fwd_node_name)
l3fwd_node_interface = l3fwd_node.get_interface(network_name=network_name)


pktgen_node = slice.get_node(name=pktgen_node_name)
pktgen_node_interface = pktgen_node.get_interface(network_name=network_name)

## Run the Experiment


### Start L3FWD Application on l3fwd-node

Open a SSH console for `l3fwd-node` and start `dpdk-l3fwd` application using the commands generated by the next cell:

In [ ]:
print(f"Open a SSH console for {l3fwd_node.get_name()} using the command: {l3fwd_node.get_ssh_command()}")

In [ ]:
print("Start DPDK L3fwd using the command: ")
print(f'cd dpdk-23.07;sudo ./build/examples/dpdk-l3fwd -a {pktgen_node_interface.get_component().get_pci_addr()[0]} -c 0x1 -n 4 -- -p 0x1 --config="(0,0,0)"  --eth-dest=0,{pktgen_node_interface.get_mac().lower()}')

### L3fwd example output

<img src="./images/l3fwd.png"><br>

### Start Pktgen Application on pktgen-node

Open a SSH console for `pktgen-node` and start `pktgen` using the commands generated by the next cell:

In [ ]:
print(f"Open a SSH console for {pktgen_node.get_name()} using the command: {pktgen_node.get_ssh_command()}")

In [ ]:
print("Start DPDK PKTGEN using the command: ")
if "rocky" in pktgen_node.get_image():
    print("export PKG_CONFIG_PATH=/usr/local/lib64/pkgconfig")
    print("cd Pktgen-DPDK")
else:
    print("cd pktgen-dpdk")
print(f"sudo ./Builddir/app/pktgen -a {pktgen_node_interface.get_component().get_pci_addr()[0]} -l 0-4 -n 3 -- -P -m [1:3].0")

After Pktgen is brought up, set up Pktgen and start traffic by running the commands generated by the next cell:

In [ ]:
print(f"set 0-1 dst mac {l3fwd_node_interface.get_mac().lower()}")
print(f"set 0-1 src mac {pktgen_node_interface.get_mac().lower()}")
print("set 0-1 size 128")
print("set 0-1 rate 50")
print("start 0")
print("start 1")

### Pktgen example output

<img src="./images/pktgen.png"><br>

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()